<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20/%D0%90%D0%B9%D0%A2%D0%B8%D0%9E%D0%BD_%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%BE%D0%B2/json_generator_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 35.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install dateparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.8 MB/s eta 0:00:00


In [ ]:
import spacy
import dateparser
from datetime import datetime, date
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# рабочая папка
work_path = '/content/drive/MyDrive/NLP_spacy'

# папка для сохранения дообученной модели
output_dir = work_path+'/new_NER4'
if not os.path.isdir(output_dir): print('Папка', output_dir, 'не найдена!')

# загузим ранее сохраненную NER-модель из папки output_dir
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

In [ ]:
# Проверка правильности распознавания названий городов

import difflib  # Модуль difflib предоставляет функции для вычисления и применения операций различия между последовательностями.

def load_city_list(file_path):  # Загружам список городов из текстового файла.
    with open(file_path, 'r', encoding='utf-8') as f:
        cities = [line.strip() for line in f]
    return cities

def find_closest_city(city, city_list):   # Находим ближайший город в списке с использованием метода наибольшего общего подпоследовательности.
    matches = difflib.get_close_matches(city, city_list, n=1, cutoff=0.6)   # n - число возвращаемых совпадений, cutoff -  минимальный порог сходства
    # get_close_matches - возвращает список ближайших совпадений, найденных в city_list
    # n - число возвращаемых совпадений, cutoff -  минимальный порог сходства
    closest_city = matches[0] if len(matches) > 0 else None
    return closest_city

def correct_city(city, city_list):    # Обработка списока городов и проверка на соответствие.

    closest_city = find_closest_city(city, city_list)
    if city == "Питер" or city == "Ленинград":        # Выявленная замена
        closest_city = "Санкт-Петербург"
    if city == "Минводы" or city == "Кавминводы"  or city == "Каменвода" or city == "Каменводы":     # Выявленная замена
        closest_city = "Минеральные Воды"
    if city == "Нижний Арктовского" or city == "Нижний Арктовский":     # Выявленная замена
        closest_city = "Нижневартовск"

    return closest_city


# Загрузка списка городов из файла
file_path = "/content/drive/MyDrive/NLP_spacy/city_list.txt"  # Путь к файлу ".txt" списка городов

all_cities = load_city_list(file_path)
# Можно добавить списки других городов, например других стран
# railway_cities = load_city_list('railway_cities.txt')
# airport_cities = load_city_list('airport_cities.txt')
# bus_cities = load_city_list('bus_cities.txt')
# all_cities = all_cities + railway_cities + airport_cities + bus_cities


In [ ]:
# служебные функции "добавить" и "умножить"
def add(count): return lambda x: x+count
def mul(count): return lambda x: x*count if x > 0 else count

# словарь вариантов токенов
startTokens = {
    "один": add(1), "одна": add(1), "одного": add(1),
    "два": add(2),  "две": add(2), "двое": add(2), "вдвоём": add(2),
    "три": add(3), "трое": add(3), "втроём": add(3),
    "четыре": add(4), "четверо": add(4), "вчетвером": add(4),
    "пять": add(5), "пятеро": add(5), "впятером": add(5),
    "шесть": add(6), "шестеро": add(6), "вшестером": add(6),
    "семь": add(7), "семеро": add(7),
    "восемь": add(8),
    "девять": add(9),

    "десяток": mul(10), "десятка": mul(10), "десятков": mul(10),

    "десять": add(10),
    "одиннадцать": add(11),
    "двенадцать": add(12),
    "тринадцать": add(13),
    "четырнадцать": add(14),
    "пятнадцать": add(15),
    "шестнадцать": add(16),
    "семнадцать": add(17),
    "восемнадцать": add(18),
    "девятнадцать": add(19),
    "двадцать": add(20),
    "тридцать": add(30),
    "сорок": add(40),
    "пятьдесят": add(50),
    "шестьдесят": add(60),
    "семьдесят": add(70),
    "восемьдесят": add(80),
    "девяносто": add(90),

    "сотня": mul(100), "сотни": mul(100), "сотен": mul(100),

    "сто": add(100),
    "двести": add(200),
    "триста": add(300),
    "четыреста": add(400),
    "пятьсот": add(500),
    "шестьсот": add(600),
    "семьсот": add(700),
    "восемьсот": add(800),
    "девятьсот": add(900),

    "тысяча": mul(1000), "тысячи": mul(1000), "тысяч": mul(1000), "тыс": mul(1000)
}

# функция определения действия по токену
def matchToken(token):
  try:
    return startTokens[token]
  except:
    if token.isdigit():
      return add(int(token))
    return add(0)


# функция для преобразования числа, заданного словами, в число
def str2num(st):

    if st.isdigit():
      return int(st)

    tokens = st.split(" ")
    value = 0
    for token in tokens:
      value = matchToken(token)(value)

    return value


In [ ]:
# Функция, которая формирует новую пустую поездку
def get_new_route():

    route = {
        'city_from': None,     # город отправления (может быть null)
        'date_from': None,     # дата отправления (может быть null)
        'date_from_t': None,   # дата отправления, если не удалось определить точное значение (может быть null)
        'time_from': None,     # время отправления (может быть null)
        'city_to': None,       # город прибытия (может быть null)
        'date_to': None,       # дата прибытия (может быть null)
        'date_to_t': None,     # дата прибытия, если не удалось определить точное значение (может быть null)
        'time_to': None,       # время прибытия (может быть null)
        'amount': None,        # количество пассажиров (минимум один)
        'transit': None,       # количество пересадок (может быть null)
#        'replace_city': None,  # города для возможных пересадок (может быть null)
        'transport': None,     # вид транспорта -  'поезд', 'самолет', 'автобус', 'паром' (может быть null)
        'company': None,       # компания-перевозчик (может быть null)
        'animal': None,        # с животными (может быть null)
        'children': None,      # количество детей (может быть null)
        'baggage': None,       # с багажом (может быть null)
        'level': None,         # класс обслуживания - 'бизнес', 'эконом', 'купе', 'плацкарт' (может быть null)
        'comfort': None,       # список дополнительных пожеланий - 'нижняя полка', 'мягкий вагон', 'с питанием', 'женское купе', 'не у туалета', 'биотуалет'  (может быть null)
        'insurance': None      # страховка (может быть null)
    }

    return route


# Функция, которая формирует json на основе выделенных сущностей
def create_json(doc, filename, transcribator='whisper(model=large)'):

    entities = [{'text': ent.text, 'lemma': ent.lemma_, 'label': ent.label_} for ent in doc.ents]

    json_data = {
        'route': [],                      # маршрут - список поездок (обязательно должен быть хотя бы один элемент)
        'all_budget': None,               # общий бюджет на весь маршрут (может быть null)
        'filename': filename,             # название транскрибированного аудиофайла
#        'real_text': None,
        'transcrib_text': doc.text,       # транскрибированный текст
        'transcribator': transcribator    # использованный транскрибатор
    }

    # настройки для парсера дат
    settings = {
        'DATE_ORDER': 'DMY',
        'PREFER_DATES_FROM': 'future'
    }

    #format_date = '%d.%m.%Y'
    format_date = '%d.%m'

    route = get_new_route()  # создаем новую поездку
    pred_route = route.copy()  # нужно определить и предыдущую поездку

    new_route = True
    ll = len(entities)
    for i in range(ll):
        entity = entities[i]
        if entity['label'] == 'LOC':  # анализ сущности LOCATION
            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if (entity_pred['label'] == 'FROM'):  # если предыдущая сущность FROM, то это город отправления
                route['city_from'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
            elif (entity_pred['label'] == 'TO'):  # если предыдущая сущность TO, то это город назначения
                route['city_to'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
            elif entity_pred['label'] == 'TRANSIT':  # если предыдущая сущность TRANSIT, то это город пересадки
                if new_route:
               #     route['replace_city'] = []
                    new_route = False
                # if route['replace_city'] is None:
                #     route['replace_city'] = []
                # route['replace_city'].append(entity['lemma'].title().replace('.', ''))
                route['transit'] = 1
            elif route['city_from'] is None:  # если предыдущая сущность не подходящая, город отправления определен, а город назначения еще не определен, то это город назначения
                route['city_from'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
            elif route['city_to'] is None:  # если предыдущая сущность не подходящая и город отправления еще не определен, то это город отправления
                route['city_to'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
        elif entity['label'] == 'DATE':  # анализ сущности DATE
            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if (entity_pred['label'] == 'ARRIVAL'):  # проверим предыдущую сущность, если она ARRIVAL, то это дата прибытия
                st = entity['lemma']
                if st.find('числ') > -1:  # если найдем "число", то заменим на месяц от текущей даты
                    st = st.replace('число', datetime.strftime(date.today(), '%B'))
                dt = dateparser.parse(st, settings=settings)
                if dt is None: dt = date.today()
                route['date_to'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_to_t'] = entity['text']
            elif route['date_from_t'] is None:  # записываем в дату отправления, если она еще не задана
                st = entity['lemma']
                if st.find('числ') > -1:  # если найдем "число", то заменим на текущий месяц
                    if pred_route['date_from'] is not None:  # если была предыдущая поездка и определена дата отправления
                        st = st.replace('число', datetime.strptime(pred_route['date_from'], format_date).strftime('%B'))
                    elif pred_route['date_to'] is not None:  # ... или определена дата прибытия
                        st = st.replace('число', datetime.strptime(pred_route['date_to'], format_date).strftime('%B'))
                    else:  # иначе возьмем месяц от текущей даты
                        st = st.replace('число', datetime.strftime(date.today(), '%B'))
                dt = dateparser.parse(st, settings=settings)
                if dt is None: dt = date.today()
                route['date_from'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_from_t'] = entity['text']
            else:  # а если дата отправления уже определена, то записываем в дату прибытия
                st = entity['lemma']
                if st.find('числ') > -1:  # если найдем "число", то заменим на текущий месяц
                    if route['date_from'] is not None:  # если определена дата отправления
                        st = st.replace('число', datetime.strptime(route['date_from'], format_date).strftime('%B'))
                    else:  # иначе возьмем месяц от текущей даты
                        st = st.replace('число', datetime.strftime(date.today(), '%B'))
                dt = dateparser.parse(st, settings=settings)
                if dt is None: dt = date.today()
                route['date_to'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_to_t'] = entity['text']
        elif entity['label'] == 'GAP':  # анализ сущности GAP
            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if i < ll-1:
                entity_next = entities[i+1]
            else:
                entity_next = entity
            if (entity_pred['label'] == 'RETURN') or (entity_next['label'] == 'RETURN'):
                # проверим предыдущую и следующую сущность и, если она RETURN, то анализ GAP пропускаем - будет выполняться при анализе RETURN
                continue
            elif route['date_from_t'] is None:  # записываем в дату отправления, если она еще не задана
                dt = dateparser.parse(entity['text'], settings=settings)
                if dt is None: dt = date.today()
                route['date_from'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_from_t'] = entity['text']
            else:  # а если дата отправления уже определена, то записываем в дату прибытия
                dt = dateparser.parse(entity['text'], settings=settings)
                if dt is None: dt = date.today()
                route['date_to'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_to_t'] = entity['text']
        elif entity['label'] == 'TIME':  # анализ сущности TIME
            if route['time_from'] is None:  # проверим время отправления, если оно еще не задано
                # дополнительно проверим даты отправления и прибытия
                # если дата отправления не задана, а дата прибытия определена, то скорее всего это время прибытия
                if (route['date_from_t'] is None) and (route['date_to_t'] is not None):
                    route['time_to'] = entity['text']
                else:  # в других случаях - время отправления
                    route['time_from'] = entity['text']
            else:  # а если время отправления уже определено, то записываем во время прибытия
                route['time_to'] = entity['text']
        elif entity['label'] == 'TRANSIT':  # здесь отлавливаем "без пересадок"
            if entity['text'].find('без') == -1: route['transit'] = 0
        elif entity['label'] == 'AMOUNT':  # кол-во пассажиров
            route['amount'] = str2num(entity['lemma'].replace('.', ''))
        elif entity['label'] == 'TRANSPORT':  # добавляем вид транспорта
            if route['transport'] is None: route['transport'] = []
            route['transport'].append(entity['lemma'].replace('.', ''))
        elif entity['label'] == 'ORG':  # добавляем вид организацию-перевозчика
            if route['company'] is None: route['company'] = []
            route['company'].append(entity['lemma'].title().replace('.', ''))
        elif entity['label'] == 'ANIMAL':  # "без животных" или есть какие-то животные
            route['animal'] = entity['text'].find('без') == -1
        elif entity['label'] == 'CHILD':  # кол-во детей, с проверкой на "без детей"
            if entity['text'].find('без') > -1:
                route['children'] = 0
            else:
                col = str2num(entity['lemma'].replace('.', ''))
                if col == 0: col = 1
                route['children'] = col
        elif entity['label'] == 'BAGG':  # "без багажа" или есть какой-то багаж
            route['baggage'] = entity['text'].find('без') == -1
        elif entity['label'] == 'LEVEL':  # добавляем уровень обслуживания
            if route['level'] is None: route['level'] = []
            route['level'].append(entity['text'])
        elif entity['label'] == 'COMFORT':  # добавляем пожелания к комфорту
            if route['comfort'] is None: route['comfort'] = []
            route['comfort'].append(entity['text'])
        elif entity['label'] == 'INSUR':  # "без страховки" или страховка есть
            route['insurance'] = entity['text'].find('без') == -1
        elif entity['label'] == 'BUDGET':  # общий бюджет
            json_data['all_budget'] = str2num(entity['lemma'].replace('.', ''))
        elif entity['label'] == 'RETURN':
            # если встречаем сущность RETURN, то формируем новую поездку, в которую записываем города отправления и прибытия из предыдущей поездки, но в обратном порядке
            # все остальные сущности просто копируем, предполагая, что все ранее определенные особенности поездки будут такими же
            # при этом даты не копируем, т.к. ясно, что обратная поездка будет позже
            # если дальше при разборе фразы будут найдены сущности характеристик поездки, то просто перезапишем
            # особенность с транзитными городами: если находим сущность TRANSIT, то обнуляем транзитные города предыдущей поездки

            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if entity_pred['label'] == 'RETURN': continue  # не обрабатываем, если несколько сущностей RETURN идет подряд

            if (route['date_from_t'] is None) and (route['date_to_t'] is None):  # если дата отправления осталась не опеределенной, то заполняем ее текущей датой
                route['date_from'] = datetime.strftime(date.today(), format_date)
                route['date_from_t'] = ''

            pred_route = route.copy()  # скопируем текущую поездку
            json_data['route'].append(route)  # добавляем текущую поездку в маршрут
            route = get_new_route()  # создаем новую поездку
            new_route = True

            # заполним некоторые параметры новой поездки на основании предыдущей
            route['city_from'] = pred_route['city_to']
            route['city_to'] = pred_route['city_from']
            route['transit'] = pred_route['transit']
            #route['replace_city'] = pred_route['replace_city']
            route['amount'] = pred_route['amount']
            route['transport'] = pred_route['transport']
            route['animal'] = pred_route['animal']
            route['children'] = pred_route['children']
            route['baggage'] = pred_route['baggage']
            route['level'] = pred_route['level']
            route['comfort'] = pred_route['comfort']
            route['insurance'] = pred_route['insurance']

            if i < ll-1:
                entity_next = entities[i+1]
            else:
                entity_next = entity
            if (entity_pred['label'] == 'GAP') or (entity_next['label'] == 'GAP'):  # проверим предыдущую и следующую сущность, если она GAP, то попробуем определить дату отправления для новой поездки
                if entity_pred['label'] == 'GAP':
                    route['date_from_t'] = entity_pred['text']
                else:
                    route['date_from_t'] = entity_next['text']
                settings_modify = settings.copy()
                # в настройки парсера дат добавляем "считать от даты..."
                if pred_route['date_from'] is not None: # если определена дата отправления
                    settings_modify['RELATIVE_BASE'] = datetime.strptime(pred_route['date_from'], format_date)
                elif pred_route['date_to'] is not None: # если определена дата прибытия
                    settings_modify['RELATIVE_BASE'] = datetime.strptime(pred_route['date_to'], format_date)
                # или останется текущая дата

                dt = dateparser.parse(route['date_from_t'], settings=settings_modify)
                if dt is None: dt = date.today()
                route['date_from'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая

    json_data['route'].append(route)  # добавляем текущую поездку в маршрут

    return json_data

In [ ]:
# тестирование инициализированной модели
#test_text = "Самолет до Москвы из Владивостока. Там нужно быть 5 мая вечером. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Можно даже через Новосибирск. Обратно 26 числа. Покажи все, что есть. Общий бюджет 10000 рублей."
#test_text = "Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей."
#test_text = "Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть. С прибытием не позже 6 утра следующего дня."

test_text = "Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря пассажир 1."

doc = nlp(test_text)

print([(w.text, w.pos_) for w in doc])
print()
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
print()
spacy.displacy.render(doc, style="ent", jupyter=True)

[('Какие', 'DET'), ('есть', 'VERB'), ('варианты', 'NOUN'), ('добраться', 'VERB'), ('15', 'ADJ'), ('декабря', 'NOUN'), ('из', 'ADP'), ('Краснодара', 'PROPN'), ('в', 'ADP'), ('Новосибирск', 'PROPN'), ('?', 'PUNCT'), ('Обратный', 'ADJ'), ('возврат', 'NOUN'), ('19', 'ADJ'), ('декабря', 'NOUN'), ('пассажир', 'NOUN'), ('1', 'NUM'), ('.', 'PUNCT')]

Entities [('15 декабря', 'DATE'), ('из', 'FROM'), ('Краснодара', 'LOC'), ('в', 'TO'), ('Новосибирск', 'LOC'), ('Обратный', 'RETURN'), ('возврат', 'RETURN'), ('19 декабря', 'DATE'), ('пассажир 1.', 'AMOUNT')]



In [ ]:
entities = [{'text': ent.text, 'lemma': ent.lemma_, 'label': ent.label_} for ent in doc.ents]
entities

[{'text': '15 декабря', 'lemma': '15 декабрь', 'label': 'DATE'},
 {'text': 'из', 'lemma': 'из', 'label': 'FROM'},
 {'text': 'Краснодара', 'lemma': 'краснодар', 'label': 'LOC'},
 {'text': 'в', 'lemma': 'в', 'label': 'TO'},
 {'text': 'Новосибирск', 'lemma': 'новосибирск', 'label': 'LOC'},
 {'text': 'Обратный', 'lemma': 'обратный', 'label': 'RETURN'},
 {'text': 'возврат', 'lemma': 'возврат', 'label': 'RETURN'},
 {'text': '19 декабря', 'lemma': '19 декабрь', 'label': 'DATE'},
 {'text': 'пассажир 1.', 'lemma': 'пассажир 1.', 'label': 'AMOUNT'}]

In [ ]:
filename = 'audio_01_01.wav'

# jsonfile = ''.join(filename.split('.')[:-1])+'.json'
# with open(jsonfile, 'w') as fp:
#     json.dump(create_json(doc, filename), fp, ensure_ascii=False, indent=4)

create_json(doc, filename)

{'route': [{'city_from': 'Краснодар',
   'date_from': '15.12',
   'date_from_t': '15 декабря',
   'time_from': None,
   'city_to': 'Новосибирск',
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': None,
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None},
  {'city_from': 'Новосибирск',
   'date_from': '19.12',
   'date_from_t': '19 декабря',
   'time_from': None,
   'city_to': 'Краснодар',
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': 1,
   'transit': None,
   'transport': None,
   'company': None,
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': 'audio_01_01.wav',
 'transcrib_text': 'Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря пассажир 1.',
 'tran

In [ ]:
name_transcribator = 'whisper(model=large)'
input_path = work_path + '/' + name_transcribator
output_path = work_path + '/' + name_transcribator + '_json'

if not os.path.isdir(input_path):
    print('Папка', input_path, 'не найдена!')
else:
    if not os.path.isdir(output_path): os.makedirs(output_path)

    # получим список файлов в папке с данными
    file_list = sorted([f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))])
    len(file_list)

In [ ]:
# сформируем список тикеров, для которых есть файлы с данными
count_files = 0
for filename in file_list:
    count_files += 1
    with open(os.path.join(input_path, filename), 'r', encoding='utf-8') as f:
        txt = f.read()
    print('#', count_files, ' ', filename, end='   ')
    doc = nlp(txt)
    js = create_json(doc, filename, transcribator=name_transcribator)
    # print('')
    # print(js)

    jsonfile = ''.join(filename.split('.')[:-1])+'.json'
    with open(os.path.join(output_path, jsonfile), 'w') as fp:
        json.dump(js, fp, ensure_ascii=False, indent=4)
    print('+')
    # print('')

# 1   audio_2023-11-20_18-04-29.txt   +
# 2   audio_2023-11-20_18-04-59.txt   +
# 3   audio_2023-11-20_18-05-06.txt   +
# 4   audio_2023-11-20_18-05-12.txt   +
# 5   audio_2023-11-20_18-05-18.txt   +
# 6   audio_2023-11-20_18-06-06.txt   +
# 7   audio_2023-11-20_18-06-10.txt   +
# 8   audio_2023-11-20_18-06-14.txt   +
# 9   audio_2023-11-20_18-06-20.txt   +
# 10   audio_2023-11-20_18-06-26.txt   +
# 11   audio_2023-11-20_18-06-31.txt   +
# 12   audio_2023-11-20_18-06-41.txt   +
# 13   audio_2023-11-20_18-06-58.txt   +
# 14   audio_2023-11-20_18-07-03.txt   +
# 15   audio_2023-11-20_18-07-09.txt   +
# 16   audio_2023-11-20_18-07-15.txt   +
# 17   audio_2023-11-20_18-07-19.txt   +
# 18   audio_2023-11-20_18-07-23.txt   +
# 19   audio_2023-11-20_18-07-27.txt   +
# 20   audio_2023-11-20_18-08-03.txt   +
# 21   audio_2023-11-20_18-08-06.txt   +
# 22   audio_2023-11-20_18-08-17.txt   +
# 23   audio_2023-11-20_18-08-21.txt   +
# 24   audio_2023-11-20_18-08-24.txt   +
# 25   audio_2023-11-20_1